In [1]:
from pathlib import Path
import shutil

In [4]:
# Define your output directory (folder containing the results of the analysis in 1_image_analysis)
PARENT_FOLDER = Path("./output")

# Define your username (subfolder name under the output directory)
USERNAME = "Andrew"

# Merge parent_folder and username into a Path
results_directory = PARENT_FOLDER / USERNAME

# Create an empty list to store the subdirectories within ./output/USERNAME
subdirectories = []

# Iterate over subfolders in the results_directory and add them to subdirectories list
for subfolder in results_directory.iterdir():
    if subfolder.is_dir():
        subdirectories.append(subfolder.name)

In [5]:
subdirectories

['P1.2023-09-29-11-56-39',
 'P1.2023-10-02-08-13-59',
 'P2.2023-09-29-12-23-28',
 'P2.2023-10-02-08-55-55',
 'P3.2023-09-29-12-51-41',
 'P3.2023-10-02-09-16-14',
 'test_data_2']

In [6]:
for subdir in subdirectories:
    # Build the path to each of the subdirectories
    subdirectory_path = Path(results_directory) / subdir

    # Check if the subdirectory exists
    if subdirectory_path.exists():
        print(subdirectory_path)

output\Andrew\P1.2023-09-29-11-56-39
output\Andrew\P1.2023-10-02-08-13-59
output\Andrew\P2.2023-09-29-12-23-28
output\Andrew\P2.2023-10-02-08-55-55
output\Andrew\P3.2023-09-29-12-51-41
output\Andrew\P3.2023-10-02-09-16-14
output\Andrew\test_data_2
